In [19]:
s = "こんにちわ　你好 hello 😳"
s

'こんにちわ\u3000你好 hello 😳'

In [20]:
len(s)

16

In [21]:
list(s.encode("utf-16"))

[255,
 254,
 83,
 48,
 147,
 48,
 107,
 48,
 97,
 48,
 143,
 48,
 0,
 48,
 96,
 79,
 125,
 89,
 32,
 0,
 104,
 0,
 101,
 0,
 108,
 0,
 108,
 0,
 111,
 0,
 32,
 0,
 61,
 216,
 51,
 222]